In [69]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import shfl
import torch
import copy
import cv2
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_data_csv
from ClassifierModel import ClassifierModel

In [70]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", folds=1, lambda_values = [0.05], batch_size=1, epochs=1, device='cpu')


In [ ]:
"""
cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()

cit2._G_dict['P'].to('cpu')

#for p in cit2._G_dict['P'].parameters():
#    print(p)

state_dict = cit2._G_dict['P'].state_dict()
new_state_dict = copy.deepcopy(state_dict)

for k in state_dict:
    print(state_dict[k])
"""

In [ ]:
"""
with torch.no_grad():
    
    for k in state_dict:
        #print(k)
        new_state_dict[k] = 2*state_dict[k]
        #print(new_state_dict[k])
    cit2._G_dict['P'].load_state_dict(new_state_dict)
    

"""
    

In [ ]:
"""
state_dict = cit2._G_dict['P'].state_dict()
for k in state_dict:
    print(state_dict[k])
"""

In [71]:
cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()

for cit in [cit1, cit2, cit3]:
    state_dict = cit._G_dict['P'].state_dict()
    k = "block1.0.weight"
    print(state_dict[k][0][0][0])

aggregator = shfl.federated_aggregator.FedAvgAggregator()
agg = aggregator.aggregate_weights([cit1.get_model_params(), cit2.get_model_params(), cit3.get_model_params()])
cit1.set_model_params(agg)

print("bbb")
state_dict = cit1._G_dict['P'].state_dict()
k = "block1.0.weight"
print(state_dict[k][0][0][0])


tensor([-0.0183,  0.0027, -0.0244, -0.0293, -0.0254,  0.0475,  0.0497, -0.0322,
        -0.0335])
tensor([-0.0339, -0.0061, -0.0563,  0.0572, -0.0450,  0.0153,  0.0257,  0.0483,
         0.0178])
tensor([ 0.0006,  0.0314,  0.0198,  0.0197,  0.0403,  0.0529, -0.0147, -0.0295,
         0.0562])
bbb
tensor([-0.0172,  0.0093, -0.0203,  0.0159, -0.0100,  0.0386,  0.0202, -0.0045,
         0.0135])


In [77]:
cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()

state_dict = cit1._classifier.state_dict()
state_dict[k][0][0][0][0] = 3
cit1._classifier.load_state_dict(state_dict)

k = "conv1.weight"
for cit in [cit1, cit2, cit3]:
    state_dict = cit._classifier.state_dict()
    print(state_dict[k][0][0][0])

state_dict[k][0][0][0][0] = 3
cit1._classifier.load_state_dict(state_dict)
    
aggregator = shfl.federated_aggregator.FedAvgAggregator()
agg = aggregator.aggregate_weights([cit1.get_model_params(), cit2.get_model_params(), cit3.get_model_params()])
cit1.set_model_params(agg)

print("bbb")
state_dict = cit1._classifier.state_dict()
k = "conv1.weight"
print(state_dict[k][0][0][0])

tensor([-0.0104, -0.0061, -0.0018,  0.0748,  0.0566,  0.0171, -0.0127])
tensor([-0.0104, -0.0061, -0.0018,  0.0748,  0.0566,  0.0171, -0.0127])
tensor([-0.0104, -0.0061, -0.0018,  0.0748,  0.0566,  0.0171, -0.0127])
bbb
tensor([ 1.9965e+00, -6.1356e-03, -1.8098e-03,  7.4841e-02,  5.6615e-02,
         1.7083e-02, -1.2694e-02])
